### Uses a very broad search (Parent term contains string 'Cardiovascular') to select a large number of trait IDs (these are later pruned)

In [1]:
# supress warnings
options(warn=-1)

In [2]:
# get date for time stamping
today=format(Sys.Date(), "%Y-%m-%d")

In [3]:
# load packages
library('gwasrapidd')
library('tidyverse')
library('glue')                     

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.1 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::n()      masks gwasrapidd::n()


In [4]:
# Get all efo_ids 
ebi<-read_csv('/home/jovyan/EBI_codes.csv',show_col_types = FALSE)
head(ebi)

Disease trait,EFO term,EFO URI,Parent term,Parent URI,EFO_ids
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Cervical cancer,cervical cancer,http://purl.obolibrary.org/obo/MONDO_0002974,Cancer,http://www.ebi.ac.uk/efo/EFO_0000616,MONDO_0002974
Clubfoot,familial clubfoot with or without associated lower limb anomalies,http://www.orpha.net/ORDO/Orphanet_199315,NR,NR,Orphanet_199315
X-12093 levels,X-12093 measurement,http://www.ebi.ac.uk/efo/EFO_0021285,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444,EFO_0021285
Alcohol dependence or heroin dependence or methamphetamine dependence,methamphetamine dependence,http://www.ebi.ac.uk/efo/EFO_0004701,Neurological disorder,http://www.ebi.ac.uk/efo/EFO_0000618,EFO_0004701
Methamphetamine dependence,methamphetamine dependence,http://www.ebi.ac.uk/efo/EFO_0004701,Neurological disorder,http://www.ebi.ac.uk/efo/EFO_0000618,EFO_0004701
X-12063 levels,X-12063 measurement,http://www.ebi.ac.uk/efo/EFO_0021283,Other measurement,http://www.ebi.ac.uk/efo/EFO_0001444,EFO_0021283


In [5]:
# get traits for Cardiovascular Disease Traits
cv_only <- ebi %>% filter(str_detect(ebi$`Parent term`, "ardiovascular") | str_detect(ebi$`Disease trait`, "ongenital")) # get a broad list of traits
efo_ids=unique(cv_only$EFO_ids)
efo_ids<-append(efo_ids,'MONDO_0005178') # Add Osteoarthritis as a negative control

In [6]:
# get metadata for those trait terms, the efo_ids in particular
efos <- get_traits(efo_id = efo_ids)
efos <- efos@traits # gets the (only) traits 'slot' of the output
print(dim(efos))
head(efos)

[1] 272   3


efo_id,trait,uri
<chr>,<chr>,<chr>
EFO_0004715,mri defined brain infarct,http://www.ebi.ac.uk/efo/EFO_0004715
EFO_0004718,vascular dementia,http://www.ebi.ac.uk/efo/EFO_0004718
EFO_0001666,aortic aneurysm,http://www.ebi.ac.uk/efo/EFO_0001666
Orphanet_136,cerebral autosomal dominant arteriopathy with subcortical infarcts and leukoencephalopathy,http://www.orpha.net/ORDO/Orphanet_136
EFO_0004745,nt-probnp measurement,http://www.ebi.ac.uk/efo/EFO_0004745
EFO_0011035,TP segment duration,http://www.ebi.ac.uk/efo/EFO_0011035


In [7]:
# Make a table of index SNPs for each trait

list_of_tibbles<-list()

for (i in seq_along(efos$trait)) {
    if (file.exists(glue('/nfs/team205/heart/EBI_GWAS/index_snps/{efos$efo_id[i]}_{efos$trait[i]}_index_SNPs.csv'))==FALSE) {
        print(glue('{efos$trait[i]}: file needs to be made'))
        variants_table<-get_variants(efo_id=efos$efo_id[i])@variants #gets variants for the trait
        variants_table<-variants_table[complete.cases(variants_table), ] # removes any variants where SNP coordinates are not known, since these are useless to us
        if (dim(variants_table)[1]>0) { # only keep traits for which there are SNPs
            variants_table$efo_id=efos$efo_id[i]
            variants_table$efo_term=efos$trait[i]
            n_SNPs<-dim(variants_table)[1]
            print(efos$efo_id[i])
            list_of_tibbles[[i]]<-variants_table
            write_csv(list_of_tibbles[[i]],glue('/nfs/team205/heart/EBI_GWAS/index_snps/{efos$efo_id[i]}_{efos$trait[i]}_index_SNPs.csv'))
        }
    } else {
        print(glue('{efos$trait[i]}: file already exists'))
    }
}

mri defined brain infarct: file needs to be made
vascular dementia: file already exists
aortic aneurysm: file already exists
cerebral autosomal dominant arteriopathy with subcortical infarcts and leukoencephalopathy: file already exists
nt-probnp measurement: file already exists
TP segment duration: file needs to be made
ST segment duration: file needs to be made
T wave duration: file needs to be made
carotid-femoral pulse wave velocity: file already exists
congestive heart failure: file already exists
atrial natriuretic factor measurement: file already exists
cardiotoxicity: file already exists
non-obstructive coronary artery disease: file already exists
tissue plasminogen activator measurement: file already exists
vascular endothelial growth factor measurement: file already exists
uric acid measurement: file already exists
early onset hypertension: file already exists
cardiac embolism: file already exists
small artery occlusion: file already exists
asymmetrical dimethylarginine measu